In [26]:
import os
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [14]:
import malaya

def combine(splitted):
    points = [f'{no + 1}. {malaya.text.function.transformer_textcleaning(s)}' for no, s in enumerate(splitted)]
    points = ' '.join(points)
    return points

In [4]:
import json

with open('keywords-headline.json') as fopen:
    data = json.load(fopen)

In [5]:
data[0]

[['ZLD Polres Sekadau',
  'Polres Sekadau aksi',
  'sembako warga panti',
  'Sekadau aksi bakti',
  'warga panti asuhan'],
 ['Ikatan alumni bintara polisi angkatan ke-27',
  'warga lantaran sulitnya ekonomi ditengah',
  'ide pemberian bantuan sembako muncul',
  '25 paket sembako berisi beras',
  'panti asuhan Harapan Bunda'],
 'Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan.']

In [18]:
combine(data[0][1])

'1. Ikatan alumni bintara polisi angkatan ke-27 2. warga lantaran sulitnya ekonomi ditengah 3. ide pemberian bantuan sembako muncul 4. 25 paket sembako berisi beras 5. panti asuhan Harapan Bunda'

In [21]:
from tqdm import tqdm

before, after = [], []
for row in tqdm(data):
    before.append(combine(row[0]))
    after.append(malaya.text.function.transformer_textcleaning(row[2]))
    before.append(combine(row[1]))
    after.append(malaya.text.function.transformer_textcleaning(row[2]))

100%|██████████| 156655/156655 [01:37<00:00, 1613.04it/s]


In [22]:
before[0], after[0]

('1. ZLD Polres Sekadau 2. Polres Sekadau aksi 3. sembako warga panti 4. Sekadau aksi bakti 5. warga panti asuhan',
 'Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan.')

In [23]:
batches = []
batch = 20000
for i in range(0, len(before), batch):
    index = min(i + batch, len(before))
    x = before[i: index]
    y = after[i: index]
    batches.append((x, y))

In [28]:
!rm -rf t5-data

In [27]:
import tensorflow as tf

for i in range(len(batches)):
    before = batches[i][0]
    after = batches[i][1]
    filename = f'headline-generator-{i}.tsv'
    with tf.io.gfile.GFile(filename, 'w') as outfile:
        for i in range(len(before)):
            outfile.write('%s\t%s\n' % (before[i], after[i]))
            
    blob = bucket.blob(f't5-headline-data/{filename}')
    blob.upload_from_filename(filename)